In [ ]:
import sales_prediction.step.pred_imgtab as pt
from sales_prediction.sales_prediction import EarlyStopping
from sales_prediction.preproc_tabular import get_tabular
from torch.optim.adamw import AdamW
from torch import nn
import torch

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni.pt'
desc_tot_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni_tot.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

modello = pt.create_model(1,1).to(device)

batch_size = 64
epochs = 30
criterion = nn.MSELoss()
optimizer = AdamW(
    [
        {"params": modello.image.parameters(), "lr": 1e-3},
        {"params": modello.tabular.parameters()},
        {"params": modello.final.parameters()}
    ],
    lr=1e-2,
) 
early_stop = EarlyStopping(7, 0.001) 

data, references, _, target = get_tabular(aggregated_path, desc_path)

train, val = pt.getDataset(references, data, target, img_path, batch_size, 1)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    pt.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = pt.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)

    if early_stp:
        print('Early Stop attivato.')
        break
    
print("Fatto!")

In [3]:
import sales_prediction.step.pred_tabular as pt
from sales_prediction.sales_prediction import EarlyStopping
from sales_prediction.preproc_tabular import get_tabular
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni.pt'
desc_tot_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni_tot.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

modello = pt.create_model(0, 0).to(device)

batch_size = 64
epochs = 40
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=0.01) 
early_stop = EarlyStopping(5, 0.001) 

data, _ , _, target = get_tabular(noneg_month, desc_tot_path)
train, val = pt.getDataset(data, target, batch_size, 1)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    pt.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = pt.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)

    if early_stp:
        print('Early Stop attivato.')
        break
    
print("Fatto!")

Sto usando cuda
Epoch 1
-------------------------------
MSE: 164125.593750  [   64/ 7998]


c:\Users\GRVRLD00P\Documents\Progetto ORS\Progetto ORS\ProgettoORS\sales_prediction\preproc_tabular.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  descrizioni = torch.l

MSE: 205027.671875  [ 6464/ 7998]
In questa epoca, l'MSE medio è 166798.8125
Validation Error: 
 Avg MSE: 157200.609809 
 Avg MAE: 282.980507 
 R2: -1.029805
 Bias: -283.384323

Epoch 2
-------------------------------
MSE: 162913.906250  [   64/ 7998]
MSE: 203679.000000  [ 6464/ 7998]
In questa epoca, l'MSE medio è 165623.078125
Validation Error: 
 Avg MSE: 156269.443721 
 Avg MAE: 282.077299 
 R2: -1.017768
 Bias: -282.476484

Epoch 3
-------------------------------
MSE: 161731.531250  [   64/ 7998]
MSE: 202381.812500  [ 6464/ 7998]
In questa epoca, l'MSE medio è 164472.453125
Validation Error: 
 Avg MSE: 155228.222295 
 Avg MAE: 280.725392 
 R2: -1.004316
 Bias: -281.096926

Epoch 4
-------------------------------
MSE: 160449.078125  [   64/ 7998]
MSE: 201285.609375  [ 6464/ 7998]
In questa epoca, l'MSE medio è 163354.515625
Validation Error: 
 Avg MSE: 153891.327112 
 Avg MAE: 279.303684 
 R2: -0.987018
 Bias: -279.658355

Epoch 5
-------------------------------
MSE: 159306.781250  